# Collect data from counties  in CA - LA / Bay

In [5]:
import os
import json
import http.client
import time
from pathlib import Path
from datetime import datetime
from urllib.parse import urlencode
from typing import Optional, Dict, List, Any
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv
import json
import pandas as pd
import re
import google.generativeai as genai
import os

from together import Together

# Load environment variables
load_dotenv('../../config/.env')

True

In [6]:

class RapidAPIFetcher:
    """Enhanced fetcher for US Real Estate Listings API with batch fetching support"""
    
    def __init__(self):
        """Initialize with API credentials"""
        self.api_key = os.getenv('RAPIDAPI_KEY')
        if not self.api_key:
            raise ValueError("RAPIDAPI_KEY not found in .env file")
        
        self.api_host = "us-real-estate-listings.p.rapidapi.com"
        self.headers = {
            'x-rapidapi-key': self.api_key,
            'x-rapidapi-host': self.api_host
        }
        
        # Create data directory
        
        project_root = Path.cwd()
        while project_root.name != "realestatepal" and project_root.parent != project_root:
            project_root = project_root.parent
        
        self.data_path = project_root / "backend_services" / "data" / "raw"
        self.data_path.mkdir(parents=True, exist_ok=True)
    
    def fetch_house_data(self, 
                        location: str,
                        offset: int = 0,
                        limit: int = 200,
                        property_type: str = "single_family",
                        beds_min: Optional[int] = None,
                        price_min: Optional[int] = None,
                        price_max: Optional[int] = None,
                        days_on: Optional[int] = None,
                        sort: str = "relevance") -> Dict[str, Any]:
        """
        Fetch a single batch of properties from RapidAPI
        
        Args:
            location: Location string (e.g., "Fulton County", "Atlanta, GA")
            offset: Pagination offset
            limit: Number of results (max 200)
            property_type: Type of property
            beds_min: Minimum bedrooms
            price_min: Minimum price
            price_max: Maximum price
            days_on: Days on market
            sort: Sort order
            
        Returns:
            Dict with metadata and listings
        """
        # Build query parameters
        params = {
            'location': location,
            'offset': offset,
            'limit': min(limit, 200),  # Ensure we don't exceed API limit
            'property_type': property_type,
            'sort': sort,
            'expand_search_radius': 1
        }
        
        # Add optional parameters
        if beds_min is not None:
            params['beds_min'] = beds_min
        if price_min is not None:
            params['price_min'] = price_min
        if price_max is not None:
            params['price_max'] = price_max
        if days_on is not None:
            params['days_on'] = days_on
        
        # Create query string
        query_string = urlencode(params)
        endpoint = f"/for-sale?{query_string}"
        
        try:
            # Make API request
            print(f"Fetching properties in {location} (offset: {offset}, limit: {params['limit']})...")
            conn = http.client.HTTPSConnection(self.api_host)
            conn.request("GET", endpoint, headers=self.headers)
            
            response = conn.getresponse()
            data = response.read()
            conn.close()
            
            # Parse JSON response
            result = json.loads(data.decode("utf-8"))
            
            # Check if we got properties
            if 'listings' not in result:
                print("No properties found in response")
                return {"metadata": {}, "listings": []}
            
            properties = result['listings']
            total_count = result.get('totalResultCount', len(properties))
            print(f"Found {len(properties)} properties (Total available: {total_count})")
            
            # Prepare data with metadata
            return {
                "metadata": {
                    "fetch_time": datetime.now().strftime("%Y%m%d_%H%M%S"),
                    "location": location,
                    "parameters": params,
                    "total_results": total_count,
                    "fetched_count": len(properties)
                },
                "listings": properties
            }
            
        except Exception as e:
            print(f"Error: {e}")
            return {"metadata": {}, "listings": []}
    
    def fetch_all_houses(self,
                        location: str,
                        property_type: str = "single_family",
                        beds_min: Optional[int] = None,
                        price_min: Optional[int] = None,
                        price_max: Optional[int] = None,
                        days_on: Optional[int] = None,
                        sort: str = "relevance",
                        delay: float = 0.5) -> Dict[str, Any]:
        """
        Fetch ALL properties by automatically handling pagination
        
        Args:
            location: Location string
            property_type: Type of property
            beds_min: Minimum bedrooms
            price_min: Minimum price
            price_max: Maximum price
            days_on: Days on market
            sort: Sort order
            delay: Delay between API calls in seconds
            
        Returns:
            Dict with all combined metadata and listings
        """
        all_listings = []
        offset = 0
        limit = 200  # API maximum
        total_fetched = 0
        batch_count = 0
        total_available = None
        
        print(f"\n{'='*60}")
        print(f"Starting batch fetch for: {location}")
        print(f"{'='*60}\n")
        
        while True:
            batch_count += 1
            print(f"\n--- Batch {batch_count} ---")
            
            # Fetch current batch
            result = self.fetch_house_data(
                location=location,
                offset=offset,
                limit=limit,
                property_type=property_type,
                beds_min=beds_min,
                price_min=price_min,
                price_max=price_max,
                days_on=days_on,
                sort=sort
            )
            
            # Check if we got data
            if not result or 'listings' not in result or len(result['listings']) == 0:
                print("No more data available")
                break
            
            current_batch = result['listings']
            
            # First batch: get total count
            if total_available is None and 'metadata' in result:
                total_available = result['metadata'].get('total_results', 0)
                print(f"\n📊 Total properties available: {total_available}")
            
            # Add to collection
            all_listings.extend(current_batch)
            total_fetched += len(current_batch)
            
            print(f"✅ Batch size: {len(current_batch)}")
            print(f"📈 Total fetched: {total_fetched}")
            
            # Check if we're done
            if len(current_batch) < limit:
                print("\n✓ Last batch received (less than limit)")
                break
            
            if total_available and total_fetched >= total_available:
                print("\n✓ All available properties fetched")
                break
            
            # Prepare for next batch
            offset += limit
            
            # Rate limiting
            time.sleep(delay)
        
        print(f"\n{'='*60}")
        print(f"✨ Fetch complete!")
        print(f"📊 Total properties fetched: {total_fetched}")
        print(f"📦 Total batches: {batch_count}")
        print(f"{'='*60}\n")
        
        # Return combined result
        return {
            "metadata": {
                "fetch_time": datetime.now().strftime("%Y%m%d_%H%M%S"),
                "location": location,
                "parameters": {
                    "location": location,
                    "property_type": property_type,
                    "beds_min": beds_min,
                    "price_min": price_min,
                    "price_max": price_max,
                    "days_on": days_on,
                    "sort": sort
                },
                "total_results": total_available or total_fetched,
                "fetched_count": total_fetched,
                "batch_count": batch_count
            },
            "listings": all_listings
        }
    
    def save_data(self, data: Dict[str, Any], filename_suffix: str = "complete") -> str:
        """
        Save fetched data to JSON and optionally CSV
        
        Args:
            data: Data dictionary with metadata and listings
            filename_suffix: Suffix for filename (e.g., "complete", "batch")
            
        Returns:
            Path to saved JSON file
        """
        if not data or 'listings' not in data:
            print("No data to save")
            return ""
        
        # Generate filename
        location = data['metadata'].get('location', 'unknown')
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        location_clean = location.replace(" ", "_").replace(",", "").lower()
        
        # Save JSON
        json_filename = f"{timestamp}_{location_clean}_{filename_suffix}.json"
        json_filepath = self.data_path / json_filename
        
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ JSON saved to: {json_filepath}")
        
        # Optional: Save as CSV
        try:
            
            if data['listings']:
                df = pd.json_normalize(data['listings'])
                csv_filename = f"{timestamp}_{location_clean}_{filename_suffix}.csv"
                csv_filepath = self.data_path / csv_filename
                df.to_csv(csv_filepath, index=False)
                print(f"✅ CSV saved to: {csv_filepath}")
        except ImportError:
            print("ℹ️ pandas not installed, skipping CSV export")
        except Exception as e:
            print(f"⚠️ Error saving CSV: {e}")
        
        return str(json_filepath)
    
    def fetch_and_save_all(self,
                          location: str,
                          property_type: str = "single_family",
                          beds_min: Optional[int] = None,
                          price_min: Optional[int] = None,
                          price_max: Optional[int] = None,
                          days_on: Optional[int] = None,
                          sort: str = "relevance") -> str:
        """
        Convenience method to fetch all data and save it
        
        Args:
            Same as fetch_all_houses
            
        Returns:
            Path to saved file
        """
        # Fetch all data
        all_data = self.fetch_all_houses(
            location=location,
            property_type=property_type,
            beds_min=beds_min,
            price_min=price_min,
            price_max=price_max,
            days_on=days_on,
            sort=sort
        )
        #dedup listings 
        # Save to file
        saved_path = self.save_data(all_data, "complete")
        
        return saved_path


In [7]:

# ========== 使用示例 - 每个县单独存储 ==========

fetcher = RapidAPIFetcher()

# Two counties in LA and 6 in bay area
locations = ["Los Angeles County, CA", "Orange County, CA", 
             "San Francisco County, CA", "Alameda County, CA", 
             "San Mateo County, CA", "Santa Clara County, CA",
             "Marin County, CA", "Contra Costa County, CA"]

# Common search parameters for all counties
search_parameters = {
    "property_type": "single_family",
    "beds_min": 2,
    "price_min": 700000,
    "price_max": 7000000,  # Focus on 700k-7m
    "days_on": 90,
    "sort": "relevance"
}

# Track overall statistics
overall_stats = {
    "fetch_time": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "total_locations": len(locations),
    "search_parameters": search_parameters,
    "location_results": {},
    "total_properties": 0,
    "saved_files": []
}

print(f"\n{'='*70}")
print("🏡 Starting Data Collection for CA Counties")
print(f"📍 Total counties: {len(locations)}")
print(f"💰 Price range: ${search_parameters['price_min']:,} - ${search_parameters['price_max']:,}")
print(f"🛏️  Min bedrooms: {search_parameters['beds_min']}")
print(f"📅 Days on market: {search_parameters['days_on']}")
print(f"{'='*70}")

# Process each county separately
for i, location in enumerate(locations, 1):
    print(f"\n{'='*50}")
    print(f"📍 Processing County {i}/{len(locations)}: {location}")
    print(f"{'='*50}")
    
    try:
        # Fetch data for this county
        location_data = fetcher.fetch_all_houses(
            location=location,
            **search_parameters
        )
        
        # Add search location identifier to each listing
        for listing in location_data['listings']:
            listing['search_location'] = location
        
        # Save individual county data
        county_name = location.replace(" County, CA", "").replace(" ", "_").lower()
        saved_file = fetcher.save_data(location_data, f"{county_name}_county")
        
        # Update overall statistics
        county_stats = {
            "fetched_count": len(location_data['listings']),
            "total_results": location_data['metadata'].get('total_results', 0),
            "batch_count": location_data['metadata'].get('batch_count', 0),
            "saved_file": saved_file
        }
        
        overall_stats['location_results'][location] = county_stats
        overall_stats['total_properties'] += county_stats['fetched_count']
        overall_stats['saved_files'].append(saved_file)
        
        print(f"\n✅ {location} completed:")
        print(f"   📊 Properties fetched: {county_stats['fetched_count']}")
        print(f"   💾 Saved to: {saved_file}")
        
    except Exception as e:
        print(f"\n❌ Error processing {location}: {str(e)}")
        overall_stats['location_results'][location] = {
            "error": str(e),
            "fetched_count": 0
        }

# Save overall summary
summary_file = fetcher.data_path / f"{overall_stats['fetch_time']}_CA_counties_summary.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(overall_stats, f, indent=2, ensure_ascii=False)

# Final summary
print(f"\n{'='*70}")
print("🎉 Data Collection Complete!")
print(f"{'='*70}")
print(f"📊 Overall Statistics:")
print(f"   🏠 Total properties collected: {overall_stats['total_properties']:,}")
print(f"   🗂️  Counties processed: {len([r for r in overall_stats['location_results'].values() if 'error' not in r])}/{len(locations)}")
print(f"   💾 Files created: {len(overall_stats['saved_files'])}")
print(f"\n📁 Individual County Results:")

for location, stats in overall_stats['location_results'].items():
    if 'error' not in stats:
        print(f"   • {location}: {stats['fetched_count']:,} properties")
    else:
        print(f"   • {location}: ❌ Error - {stats['error']}")

print(f"\n📋 Summary saved to: {summary_file}")
print(f"📂 All files saved in: {fetcher.data_path}")
print(f"{'='*70}")

# List all saved files for easy reference
print(f"\n📄 Generated Files:")
for file_path in overall_stats['saved_files']:
    file_name = Path(file_path).name
    file_size = Path(file_path).stat().st_size / (1024*1024)  # Convert to MB
    print(f"   • {file_name} ({file_size:.1f} MB)")
print(f"   • {Path(summary_file).name} (Summary)")
print(f"\n💡 Tip: Each county's data is now stored separately, making it easier to manage and upload to GitHub!")


🏡 Starting Data Collection for CA Counties
📍 Total counties: 8
💰 Price range: $700,000 - $7,000,000
🛏️  Min bedrooms: 2
📅 Days on market: 90

📍 Processing County 1/8: Los Angeles County, CA

Starting batch fetch for: Los Angeles County, CA


--- Batch 1 ---
Fetching properties in Los Angeles County, CA (offset: 0, limit: 200)...
Found 200 properties (Total available: 10243)

📊 Total properties available: 10243
✅ Batch size: 200
📈 Total fetched: 200

--- Batch 2 ---
Fetching properties in Los Angeles County, CA (offset: 200, limit: 200)...
Found 200 properties (Total available: 10243)
✅ Batch size: 200
📈 Total fetched: 400

--- Batch 3 ---
Fetching properties in Los Angeles County, CA (offset: 400, limit: 200)...
Found 200 properties (Total available: 10243)
✅ Batch size: 200
📈 Total fetched: 600

--- Batch 4 ---
Fetching properties in Los Angeles County, CA (offset: 600, limit: 200)...
Found 200 properties (Total available: 10243)
✅ Batch size: 200
📈 Total fetched: 800

--- Batch 5 --